In [1]:
import os
import sys

In [2]:
nb_dir = 'Users/kahotisthammer/programs/cvtkpy'
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
import re
from collections import defaultdict, Counter
from itertools import groupby, chain

In [4]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import statsmodels.api as sm
import allel as al

In [5]:
from cvtk.cvtk import TemporalFreqs, TiledTemporalFreqs
from cvtk.cov import stack_temporal_covariances
import cvtk.variant_files as vf
from cvtk.variant_files import VCFFile
from cvtk.gintervals import GenomicIntervals, GenomicInterval
from cvtk.pca import FreqPCA
from cvtk.plots import rep_plot_pca, correction_diagnostic_plot
from cvtk.utils import integerize, integerize_alternate
from cvtk.utils import extract_empirical_nulls_diagonals, extract_temporal_cov_diagonals
from cvtk.cov import stack_replicate_covariances, stack_temporal_covariances, stack_temporal_covs_by_group, cov_labels
from cvtk.cov import temporal_replicate_cov
from cvtk.bootstrap import block_bootstrap, cov_estimator, bootstrap_ci

/Users/kahotisthammer/miniconda3/lib/python3.9/site-packages/cvtkpy-0.0.1-py3.9.egg/cvtk/gintervals.py:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/Users/kahotisthammer/miniconda3/lib/python3.9/site-packages/cvtkpy-0.0.1-py3.9.egg/cvtk/gintervals.py:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?


In [6]:
from cvtk.G import G_estimator

In [7]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
mpl.rcParams['figure.figsize'] = (8.0, 4.0)
mpl.rcParams['figure.dpi'] = 200

## Load in Data

Load in the cleaned and combined metadata created in the Herring scripts. 


In [8]:
md = pd.read_csv('PH_Samples_TempCov.csv')

In [9]:
# divide into each population
md1=md[0:4]
md2=md[5:8]
md3=md[8:12]

In [28]:
md1


,time,pop,year,abbrv_year,sample,real
0,0,PWS,1991,91,PWS91,True
1,1,PWS,1996,96,PWS96,True
2,2,PWS,2006,6,PWS06,True
3,3,PWS,2017,17,PWS17,True


## Load in VCF file:

In [10]:
#PWS 
vcf1 = VCFFile('../Data/vcf/3pops_MD2000_min417_PWS_maf05.vcf.gz')

reading file '../Data/vcf/3pops_MD2000_min417_PWS_maf05.vcf.gz'...
file '../Data/vcf/3pops_MD2000_min417_PWS_maf05.vcf.gz' loaded.
total time to load VCF file: 0.381882115205129 mins.


In [12]:
#SS
vcf2 = VCFFile('../Data/vcf/3pops_MD2000_min417_SS_maf05.vcf.gz')

reading file '../Data/vcf/3pops_MD2000_min417_SS_maf05.vcf.gz'...
file '../Data/vcf/3pops_MD2000_min417_SS_maf05.vcf.gz' loaded.
total time to load VCF file: 0.3015035629272461 mins.


In [13]:
#TB
vcf3 = VCFFile('../Data/vcf/3pops_MD2000_min417_TB_maf05.vcf.gz')

reading file '../Data/vcf/3pops_MD2000_min417_TB_maf05.vcf.gz'...
file '../Data/vcf/3pops_MD2000_min417_TB_maf05.vcf.gz' loaded.
total time to load VCF file: 0.4261385679244995 mins.


In [14]:
vcf1.geno_mat.shape

(351820, 232, 2)

## Group them into each time period 

In [15]:
def parse_sample(x):
    "Parse out the sample metadata from the VCF."
    ind, pop, year = re.match(r'(?P<ind>[^_]+)_(?P<pop>[A-Z]+)(?P<year>[0-9]+)', x).groups()
    sample = pop + year
    return (ind, pop, year, sample)

In [16]:
vcf_pws = pd.DataFrame([parse_sample(str(x)) for x in vcf1.samples],
                     columns = ['ind', 'pop', 'year', 'sample'])
vcf_ss = pd.DataFrame([parse_sample(str(x)) for x in vcf2.samples],
                     columns = ['ind', 'pop', 'year', 'sample'])
vcf_tb = pd.DataFrame([parse_sample(str(x)) for x in vcf3.samples],
                     columns = ['ind', 'pop', 'year', 'sample'])

In [17]:
subpops1 = defaultdict(list)
for i, sample in enumerate(vcf_pws['sample']):
    subpops1[sample].append(i)

In [18]:
subpops2 = defaultdict(list)
for i, sample in enumerate(vcf_ss['sample']):
    subpops2[sample].append(i)

In [19]:
subpops3 = defaultdict(list)
for i, sample in enumerate(vcf_tb['sample']):
    subpops3[sample].append(i)

In [20]:
counts_mat1 = vcf1.count_alleles_subpops(subpops1)
counts_mat2 = vcf2.count_alleles_subpops(subpops2)
counts_mat3 = vcf3.count_alleles_subpops(subpops3)

/Users/kahotisthammer/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [21]:
freq_mat_all1 = vcf1.calc_freqs()
freq_mat_all2 = vcf2.calc_freqs()
freq_mat_all3 = vcf3.calc_freqs()

In [22]:
depths_mat_all1 = vcf1.N.astype('f')
depths_mat_all2 = vcf2.N.astype('f')
depths_mat_all3 = vcf3.N.astype('f')

In [23]:
ndiploids1 = {k:len(subpops1[k]) for k in vcf1.subpops}
ndiploids2 = {k:len(subpops2[k]) for k in vcf2.subpops}
ndiploids3 = {k:len(subpops3[k]) for k in vcf3.subpops}

In [33]:
ndiploids1 = {k:len(subpops1[k]) for k in vcf1.subpops}

In [34]:
ndiploids1['PWS06'] = ndiploids1['PWS07']
del ndiploids1['PWS07']
ndiploids1

{'PWS17': 56, 'PWS91': 58, 'PWS96': 72, 'PWS06': 46}

In [25]:
subpops_lkup1 = {sp:i for i, sp in enumerate(vcf1.subpops)}
subpops_lkup2 = {sp:i for i, sp in enumerate(vcf2.subpops)}
subpops_lkup3 = {sp:i for i, sp in enumerate(vcf3.subpops)}

In [35]:
subpops_lkup1['PWS06'] = subpops_lkup1['PWS07']
del subpops_lkup1['PWS07']
subpops_lkup1

{'PWS17': 1, 'PWS91': 2, 'PWS96': 3, 'PWS06': 0}

In [36]:
# No of loci is same for all
nloci= freq_mat_all1.shape[1]
nloci

351820

In [37]:
#order by years based on 'md' file

#1. PWS
pws_freqs = []
pws_depths = []
pws_samples = []
pws_ndiploids = []

for row in md1.itertuples():
    sample = row.sample
    freqs = freq_mat_all1[subpops_lkup1[sample], :]
    depths = depths_mat_all1[subpops_lkup1[sample], :]        
    ndips = ndiploids1[sample]
    pws_freqs.append(freqs)
    pws_depths.append(depths)
    pws_samples.append((row.pop, row.year))
    pws_ndiploids.append(ndips)
    
pws_full_depths_mat = np.row_stack(pws_depths)
pws_freq_mat = np.row_stack(pws_freqs)

In [38]:
#2. SS
ss_freqs = []
ss_depths = []
ss_samples = []
ss_ndiploids = []

for row in md2.itertuples():
    sample = row.sample
    if not row.real:
        print(sample + " is a NA sample")
        freqs = np.empty((nloci))
        freqs[:] = np.nan
        depths = np.empty((nloci))
        depths[:] = np.nan
        ndips = 0   
    else:
        freqs = freq_mat_all2[subpops_lkup2[sample], :]
        depths = depths_mat_all2[subpops_lkup2[sample], :]        
        ndips = ndiploids2[sample]
    ss_freqs.append(freqs)
    ss_depths.append(depths)
    ss_samples.append((row.pop, row.year))
    ss_ndiploids.append(ndips)
    
ss_full_depths_mat = np.row_stack(ss_depths)
ss_freq_mat = np.row_stack(ss_freqs)

In [39]:
#3. TB
tb_freqs = []
tb_depths = []
tb_samples = []
tb_ndiploids = []

for row in md3.itertuples():
    sample = row.sample
    if not row.real:
        print(sample + " is a NA sample")
        freqs = np.empty((nloci))
        freqs[:] = np.nan
        depths = np.empty((nloci))
        depths[:] = np.nan
        ndips = 0   
    else:
        freqs = freq_mat_all3[subpops_lkup3[sample], :]
        depths = depths_mat_all3[subpops_lkup3[sample], :]        
        ndips = ndiploids3[sample]
    tb_freqs.append(freqs)
    tb_depths.append(depths)
    tb_samples.append((row.pop, row.year))
    tb_ndiploids.append(ndips)
    
tb_full_depths_mat = np.row_stack(tb_depths)
tb_freq_mat = np.row_stack(tb_freqs)

## Building the Temporal Freqs Object

First, we need to build the tiled `TemporalFreqs` object. We load in the sequence lengths.

In [41]:
sl_d = pd.read_csv('../data/vcf/chr_sizes.bed', delimiter='\t', names=['chrom', 'start', 'end'], header=None)

seqlens = dict(zip(sl_d['chrom'].values, sl_d['end'].values))

In [42]:
#100,000 interval 
tile_width = int(1e5)
tiles = GenomicIntervals.from_tiles(seqlens, width=tile_width)

In [43]:
gi = vcf1.build_gintervals()

In [44]:
#check the samples
list(zip(pws_samples, pws_ndiploids))

[(('PWS', 1991), 58),
 (('PWS', 1996), 72),
 (('PWS', 2006), 46),
 (('PWS', 2017), 56)]

## Build the frequncy objects with 100,000 tile size

In [45]:
pws_d = TiledTemporalFreqs(tiles, freqs=pws_freq_mat, depths=pws_full_depths_mat, diploids=pws_ndiploids, gintervals=gi, samples=pws_samples)

In [46]:
ss_d = TiledTemporalFreqs(tiles, freqs=ss_freq_mat, depths=ss_full_depths_mat, diploids=ss_ndiploids, gintervals=gi, samples=ss_samples)
tb_d = TiledTemporalFreqs(tiles, freqs=tb_freq_mat, depths=tb_full_depths_mat, diploids=tb_ndiploids, gintervals=gi, samples=tb_samples)

## Calculate the genome-wide covariance for each population

In [47]:

gw_covs1 = pws_d.calc_cov() 
gw_covs2 = ss_d.calc_cov() 
gw_covs3 = tb_d.calc_cov() 


In [52]:
stack_temporal_covariances(gw_covs1, pws_d.R, pws_d.T).T[0]

array([[ 0.00030385,  0.00072014, -0.00103058],
       [ 0.00072014,  0.00011922, -0.00090018],
       [-0.00103058, -0.00090018,  0.00231046]])

In [51]:
# SS has only 1 covariance here
stack_temporal_covariances(gw_covs2, ss_d.R, ss_d.T).T[0]

array([[ 0.00327437, -0.00323847],
       [-0.00323847,  0.00302079]])

In [53]:
stack_temporal_covariances(gw_covs3, tb_d.R, tb_d.T).T[0]

array([[ 0.00287723, -0.00043366, -0.00032577],
       [-0.00043366,  0.00324222, -0.00197734],
       [-0.00032577, -0.00197734,  0.00320491]])

In [39]:
# create labels for plots, for temporal covariances
stacked_temp_labs_pws = stack_temporal_covariances(cov_labels(pws_d.R, pws_d.T, pws_d.samples, lab_var=True), pws_d.R, pws_d.T)
stacked_temp_labs_pws.T

array([[['var(PWS: 1996-1991)', 'cov(PWS: 1996-1991, PWS: 2006-1996)',
         'cov(PWS: 1996-1991, PWS: 2017-2006)'],
        ['cov(PWS: 2006-1996, PWS: 1996-1991)', 'var(PWS: 2006-1996)',
         'cov(PWS: 2006-1996, PWS: 2017-2006)'],
        ['cov(PWS: 2017-2006, PWS: 1996-1991)',
         'cov(PWS: 2017-2006, PWS: 2006-1996)', 'var(PWS: 2017-2006)']]],
      dtype='<U35')

## Bootstrapping Temporal Covs to obtain 95% CIs

In [54]:
# This calcuates the 'pivot' based confidence intervals (and they are off)
gw_covs_cis = pws_d.bootstrap_cov(B=5000, progress_bar=True, average_replicates=False)

/Users/kahotisthammer/miniconda3/lib/python3.9/site-packages/cvtkpy-0.0.1-py3.9.egg/cvtk/cov.py:316: RuntimeWarning: Mean of empty slice
/Users/kahotisthammer/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:495: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/Users/kahotisthammer/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/Users/kahotisthammer/miniconda3/lib/python3.9/site-packages/cvtkpy-0.0.1-py3.9.egg/cvtk/cov.py:348: RuntimeWarning: Degrees of freedom <= 0 for slice
/Users/kahotisthammer/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2680: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/Users/kahotisthammer/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2680: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/Users/kahotisthammer/minicon

bootstraps:   0%|          | 0/5000 [00:00<?, ?it/s]

In [55]:
# Modify the function to return the 'straps' to manually calculate CIs
def bootstrap_cov2(d, B, alpha=0.05, keep_seqids=None, progress_bar=False,
                      average_replicates=True, return_straps=False, ci_method='pivot',
                      use_bs_estimate=False,
                      **kwargs):
        """
        Bootstrap whole covaraince matrix.
        Params:
           - B: number of bootstraps
           - alpha: α level
           - keep_seqids: which seqids to include in bootstrap; if None, all are used.
           - return_straps: whether to return the actual bootstrap vectors.
           - ci_method: 'pivot' or 'percentile'
           - **kwargs: passed to calc_cov() and calc_cov_by_tile()

        """
        # our statistic:
        cov = d.calc_cov(keep_seqids=keep_seqids, **kwargs)
        if average_replicates:
            cov = np.nanmean(stack_temporal_covariances(cov, d.R, d.T), axis=2)
        # the filter of seqids is done at this step
        covs, het_denoms = d.calc_cov_by_tile(return_ratio_parts=True,
                                                 keep_seqids=keep_seqids, **kwargs)
        covs, het_denoms = np.stack(covs), np.stack(het_denoms)
        if use_bs_estimate:
            cov = None
        return block_bootstrap_ratio_averages2(covs, het_denoms,
                                              block_indices=d.tile_indices,
                                              estimator=cov_estimator,
                                              ci_method=ci_method,
                                              statistic=cov,
                                              B=B, alpha=alpha,
                                              progress_bar=progress_bar,
                                              return_straps=return_straps,
                                              # kwargs passed directly to cov_estimator
                                              average_replicates=average_replicates,
                                              R=d.R, T=d.T)
                                              
                                                                                            
                                              
                                              
def block_bootstrap_ratio_averages2(blocks_numerator, blocks_denominator,
                                   block_indices, B, estimator=np.divide,
                                   statistic=None,
                                   alpha=0.05, return_straps=False,
                                   ci_method='pivot', progress_bar=False, **kwargs):
    """
    This block bootstrap is used often for quantities we need to calculate that are
    ratios of expectations, e.g. standardized temporal covariance (cov(Δp_t, Δp_s) / p_t(1-p_t))
    and G, both of which are expectations over loci. We use the linearity of expectation
    to greatly speed up the block bootstrap procedure.

    We do so by pre-calculating the expected numerator and denominator for each block,
    and then take a weighted average over the bootstrap sample for each the numerator and
    denominator, and then take the final ratio.

    It's assumed that blocks_numerator and blocks_denominator are both multidimension arrays
    with the first dimension being the block (e.g. tile) dimension.
    """
    That = statistic   # for clarity: read T-hat
    if progress_bar:
        B_range = tnrange(int(B), desc="bootstraps")
    else:
        B_range = range(int(B))

    assert(len(blocks_numerator) == len(blocks_denominator))
    blocks = np.arange(len(blocks_numerator), dtype='uint32')

    # Calculate the weights
    weights = np.array([len(x) for x in block_indices])

    # number of samples in resample
    nblocks = len(blocks)
    straps = list()

    for b in B_range:
        bidx = np.random.choice(blocks, size=nblocks, replace=True)
        exp_numerator = weighted_mean(blocks_numerator[bidx, ...], weights=weights[bidx])
        exp_denominator = weighted_mean(blocks_denominator[bidx, ...], weights=weights[bidx])
        stat = estimator(exp_numerator, exp_denominator, **kwargs)
        straps.append(stat)
    straps = np.stack(straps)
    if That is None:
        That = np.mean(straps, axis=0)
    if return_straps:
        return That, straps, weights
    return That, straps

def weighted_mean(array, weights, axis=0):
    """
    Weighted mean for a block of resampled temporal covariance matrices.
    This uses masked_array since nothing in numpy can handle ignoring nans and
    weighted averages.
    """
    # mask the covariance matrix, since ma.average is the only nanmean
    # in numpy that takes weights
    array_masked = np.ma.masked_invalid(array)
    return np.ma.average(array_masked, axis=axis, weights=weights).data




## 1. PWS

In [56]:
that, straps, weights = bootstrap_cov2(d=pws_d, B=5000, return_straps=True, average_replicates=False)

In [66]:
# CI = Z*SD(straps)
sd=np.std(straps, axis=0)
sd

array([[0.00015284, 0.00011545, 0.00011832],
       [0.00011545, 0.0001668 , 0.00014805],
       [0.00011832, 0.00014805, 0.00018018]])

In [68]:
from scipy.stats import norm
alpha=0.05
z = norm.ppf(1-alpha/2)
z

1.959963984540054

In [69]:
ci=z*sd

In [70]:
np.savetxt("PWS_CIs_100kwindow.csv", ci, delimiter=",")

## 2. SS

In [61]:
that, straps2, weights = bootstrap_cov2(d=ss_d, B=5000, return_straps=True, average_replicates=False)

/Users/kahotisthammer/miniconda3/lib/python3.9/site-packages/cvtkpy-0.0.1-py3.9.egg/cvtk/cov.py:316: RuntimeWarning: Mean of empty slice
/Users/kahotisthammer/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:495: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/Users/kahotisthammer/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/Users/kahotisthammer/miniconda3/lib/python3.9/site-packages/cvtkpy-0.0.1-py3.9.egg/cvtk/cov.py:348: RuntimeWarning: Degrees of freedom <= 0 for slice
/Users/kahotisthammer/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2680: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/Users/kahotisthammer/miniconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2680: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/Users/kahotisthammer/minicon

In [62]:
straps2.shape

(5000, 2, 2)

In [71]:
sd=np.std(straps2, axis=0)
sd

array([[0.00018779, 0.00015982],
       [0.00015982, 0.00021167]])

In [72]:
sd=np.std(straps2, axis=0)
ci=z*sd
np.savetxt("SS_CIs_100kwindow.csv", ci, delimiter=",")

## 3. TB

In [73]:
that, straps3, weights = bootstrap_cov2(d=tb_d, B=5000, return_straps=True, average_replicates=False)

In [74]:
sd=np.std(straps3, axis=0)
ci=z*sd
np.savetxt("TB_CIs_100kwindow.csv", ci, delimiter=",")